# RPH Project - Quick start

This section aims provides an explanation of how to build and solve a problem using RPH.jl by solving a toy problem. The equivalent script and ijulia notebook can be found in the `example` folder.

#### Installation
RPH.jl is a pure julia package. It can be installed from julia by using the built-in package manager:
```julia
using Pkg
Pkg.add("https://github.com/yassine-laguel/RPH.jl")
```

#### Getting solvers
RPH depends on other solvers to optimize the subproblems. All solvers interfaced with JuMP, the julia mathematical programming language, can be used in RPH, a list of which can be found at [JuMP's documentation](http://www.juliaopt.org/JuMP.jl/v0.19.0/installation/#Getting-Solvers-1).

Note that all algorithms layout subproblem with quadratic objectives. Default subproblem solver is the interior point algorithm Ipopt.


In [1]:
using Distributed
workers() == Vector([1]) && addprocs(3)     # add 3 workers besides master

@everywhere push!(LOAD_PATH, joinpath(pwd(), ".."))  ## TODO: remove when package installed via package manager

@everywhere using JuMP, RPH
using DataStructures, LinearAlgebra, GLPK
workers()

3-element Array{Int64,1}:
 2
 3
 4

## Laying out a problem
We take the following problem as example:

\begin{aligned}
\underset{x}{\text{minimize}}\quad
& \sum_{t=1}^T C e_t + y_t \\
\text{s.t.}\quad
& q_t, y_t, e_t \ge 0 \\
& q_t \le W \\
& e_t+y_t \ge D \\
& q_1 = \bar{r}-y_1 \\
& q_t = q_{t-1}+r[\xi_t]-y_t, \; t = 2, \ldots, T.
\end{aligned}


where $C = 5$, $W = 8$, $D = 6$, $r = [2, 10]$. A scenario is defined by $(\xi_t)_{t=2, \ldots, T}$, for $\xi_t\in\{1,2\}$.

### Representing a scenario

A scenario is represented by the following structure:

In [2]:
@everywhere struct HydroThermalScenario <: AbstractScenario
    weather::Vector{Int}
end

Here, the attribut `weather` will hold one realisation of $(\xi_t)_{t=2, \ldots, T}$.

Along with this scenario structure, the function laying out the scenario objective function $f_s$ needs to be defined.
It takes as input the JuMP model that will hold $f_s$, an instance of the previously defined scenario structure, and the identifier of the scenario. 

In [3]:
@everywhere function build_fs!(model::JuMP.Model, s::HydroThermalScenario, id_scen::ScenarioId)
    C = 5
    W = 8
    D = 6
    rain = [2, 10]

    T = length(s.weather)+1
    Y = @variable(model, [1:3*T], base_name="y_s$id_scen")

    q = [Y[1+3*k] for k in 0:T-1]
    y = [Y[2+3*k] for k in 0:T-1]
    e = [Y[3+3*k] for k in 0:T-1]

    ## State variables constraints
    @constraint(model, Y[:] .>= 0)      # positivity constraint
    @constraint(model, q .<= W)         # reservoir max capacity
    @constraint(model, e .+ y .>= D)    # meet demand
    
    ## Dynamic constraints
    @constraint(model, q[1] == sum(rain)/length(rain) - y[1])
    @constraint(model, [t=2:T], q[t] == q[t-1] - y[t] + rain[s.weather[t-1]+1])
    
    objexpr = C*sum(e) + sum(y)

    return Y, objexpr, []
end


!!! note

    - The last item returned by the function should be the reference of constraints used to build the objective, none here. Such constraints can appear when modelling a ``\max(u, v)`` in the objective as a variable ``m``, under the linear constraints ``m\ge u`` and ``m\ge v``.


### Representing the scenario tree
The scenario tree represents the stage up to which scenarios are equal.

If the problem scenario tree is a [perfect *m*-ary tree](https://en.wikipedia.org/wiki/M-ary_tree#Types_of_m-ary_trees), it can be built using a buit-in function:
```julia
scenariotree = ScenarioTree(; depth=T, nbranching=2)
```

If the tree is not regular, or quite simple, it can be built by writing specifically the partition of equivalent scenarios per stage. A simple exmaple would be:
```julia
stageid_to_scenpart = [
    OrderedSet([BitSet(1:3)]),                      # Stage 1
    OrderedSet([BitSet(1), BitSet(2:3)]),           # Stage 2
    OrderedSet([BitSet(1), BitSet(2), BitSet(3)]),  # Stage 3
]
```
!!! note
    However this method is not efficient, and whenever possible, builtin functions should be priviledged.

### Building the `Problem`


In [4]:
scenid_to_weather(scen_id, T) = return [mod(floor(Int, scen_id / 2^i), 2) for i in T-1:-1:0]

T = 5
nbranching = 2

p = 0.5

nscenarios = 2^(T-1)
scenarios = HydroThermalScenario[ HydroThermalScenario( scenid_to_weather(scen_id, T-1) ) for scen_id in 0:nscenarios-1]
probas = [ prod(v*p + (1-v)*(1-p) for v in scenid_to_weather(scen_id, T-1)) for scen_id in 1:nscenarios ]

stage_to_dim = [3*i-2:3*i for i in 1:T]
scenariotree = ScenarioTree(; depth=T, nbranching=2)


pb = Problem(
    scenarios::Vector{HydroThermalScenario},
    build_fs!::Function,
    probas::Vector{Float64},
    nscenarios::Int,
    T::Int,
    stage_to_dim::Vector{UnitRange{Int}},
    scenariotree::ScenarioTree,
)

Multi-stage problem with:
 - #scenarios:   16
 - #stages   :   5
 - #dims     :   15


## Solving a problem

### Explicitly solving the problem

In [8]:
y_direct = solve_direct(pb)
println("\nDirect solve output is:")
display(y_direct)
@show objective_value(pb, y_direct);

16×15 Array{Float64,2}:
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  0.0   2.0  4.0  0.0   2.0  4.0
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  0.0   2.0  4.0  4.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  4.0   6.0  0.0  0.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  0.0  2.0  4.0  4.0   6.0  0.0  8.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  0.0   6.0  0.0  0.0   2.0  4.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  0.0   6.0  0.0  4.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  8.0   6.0  0.0  4.0   6.0  0.0
 0.0  6.0  0.0  0.0  2.0  4.0  4.0  6.0  0.0  8.0   6.0  0.0  8.0  10.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  0.0   2.0  4.0  0.0   2.0  4.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  0.0   2.0  4.0  4.0   6.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  4.0   6.0  0.0  0.0   6.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  0.0  6.0  0.0  4.0   6.0  0.0  8.0   6.0  0.0
 0.0  6.0  0.0  4.0  6.0  0.0  8.0  6.0  0.0  4.0   

--------------------------------------------------------
--- Direct solve
--------------------------------------------------------
Building global model...
Laying out nonanticipatory constraints...
Optimization... Done.
termination_status(model) = OPTIMAL::TerminationStatusCode = 1
primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1

Direct solve output is:
objective_value(pb, y_direct) = 50.0


### Solving with Progressive Hedging

In [10]:
y_PH = solve_progressivehedging(pb, ϵ_primal=1e-4, ϵ_dual=1e-4, printstep=5)
println("\nSequential solve output is:")
display(y_PH)
@show objective_value(pb, y_PH);

--------------------------------------------------------
--- Progressive Hedging
--------------------------------------------------------
 it   primal res        dual res            dot(x,u)   objective
  0   2.8475151052e+00  3.1639056725e-01   -3.886e-16  5.4372518198585304e+01
  5   2.3662176642e-01  2.6291307380e-02   -2.498e-15  5.1675819821672491e+01
 10   4.0017374100e-02  4.4463749000e-03   -3.775e-15  5.1053561445073541e+01
 15   2.1131800407e-02  2.3479778230e-03    2.776e-17  5.0722758171873998e+01
 20   1.1140708711e-02  1.2378565235e-03    7.494e-16  5.0394139542961135e+01
 25   4.3963340492e-03  4.8848156102e-04   -8.882e-16  5.0111886977194878e+01
 30   1.5924595593e-03  1.7693995104e-04   -9.534e-15  5.0080059293333328e+01
 35   8.4434822816e-04  9.3816469796e-05   -8.521e-15  5.0062604668416576e+01
 40   4.4768730919e-04  4.9743034354e-05   -1.237e-14  5.0045193974878224e+01
 45   2.3737117524e-04  2.6374575027e-05   -1.321e-14  5.0027806574550716e+01
 50   1.258581095

16×15 Array{Float64,2}:
 5.05888e-8  6.0  -8.5113e-9  …  -9.37352e-9   2.0       4.0       
 5.05888e-8  6.0  -8.5113e-9      4.0          6.0      -9.37352e-9
 5.05888e-8  6.0  -8.5113e-9      6.88886e-8   6.0      -9.36847e-9
 5.05888e-8  6.0  -8.5113e-9      8.0          6.0      -9.48573e-9
 5.05888e-8  6.0  -8.5113e-9     -9.37352e-9   2.0       4.0       
 5.05888e-8  6.0  -8.5113e-9  …   4.0          6.0      -9.37352e-9
 5.05888e-8  6.0  -8.5113e-9      3.94425      6.0      -9.39859e-9
 5.05888e-8  6.0  -8.5113e-9      8.0          9.94464  -9.49882e-9
 5.05888e-8  6.0  -8.5113e-9     -9.37352e-9   2.0       4.0       
 5.05888e-8  6.0  -8.5113e-9      4.0          6.0      -9.37352e-9
 5.05888e-8  6.0  -8.5113e-9  …   2.25904e-7   6.0      -9.3718e-9 
 5.05888e-8  6.0  -8.5113e-9      8.0          6.0      -9.50392e-9
 5.05888e-8  6.0  -8.5113e-9     -8.64962e-9   6.0      -8.85243e-9
 5.05888e-8  6.0  -8.5113e-9      8.0          6.0      -9.4963e-9 
 5.05888e-8  6.0  -8.511

 53   9.7647796779e-05  1.0849755198e-05   -2.107e-14  5.0003483555173190e+01
Computation time: 4.482162952423096

Sequential solve output is:
objective_value(pb, y_PH) = 50.00348355517319


### Solving with Randomized Progressive Hedging

In [11]:
y_sync = solve_randomized_sync(pb, maxtime=5, printstep=50)
println("\nSynchronous solve output is:")
display(y_sync)
@show objective_value(pb, y_sync);

--------------------------------------------------------
--- Randomized Progressive Hedging - synchronous
--------------------------------------------------------
   it   global residual   objective
Initialisation... done
   16   0.0000000000e+00  5.4372518198585304e+01
   50   9.9081745030e-02  5.5736655344286838e+01
  100   5.2286191599e-01  5.2447537541916191e+01
  150   4.3592328730e-01  5.1300904833062475e+01
  200   2.9490687921e-02  5.1036683836536916e+01
  250   8.5776283508e-05  5.0852625779766853e+01
  300   2.7784843866e-03  5.0598205597213102e+01
  350   1.1025107132e-04  5.0473291894075089e+01
  400   4.3721411816e-06  5.0306823190569858e+01
  450   1.5160347595e-04  5.0148616336546766e+01
  500   1.6477341766e-01  5.0087980462545559e+01
  550   1.6337565968e-07  5.0070941203983907e+01
  600   1.5413388658e-06  5.0066637820960437e+01
  650   1.0863979246e-07  5.0057230540737500e+01
  700   1.8442037848e-07  5.0037221839199191e+01
  750   9.9992303184e-02  5.003530804465270

16×15 Array{Float64,2}:
 5.41305e-8  6.0  -5.06122e-9  …  -9.37352e-9   2.0       4.0       
 5.41305e-8  6.0  -5.06122e-9      4.0          6.0      -9.37352e-9
 5.41305e-8  6.0  -5.06122e-9      8.76107e-8   6.0      -9.37035e-9
 5.41305e-8  6.0  -5.06122e-9      8.0          6.0      -9.47573e-9
 5.41305e-8  6.0  -5.06122e-9     -9.37352e-9   2.0       4.0       
 5.41305e-8  6.0  -5.06122e-9  …   4.0          6.0      -9.37352e-9
 5.41305e-8  6.0  -5.06122e-9      3.74202      6.0      -9.40835e-9
 5.41305e-8  6.0  -5.06122e-9      8.0          9.72537  -9.49882e-9
 5.41305e-8  6.0  -5.06122e-9     -9.37352e-9   2.0       4.0       
 5.41305e-8  6.0  -5.06122e-9      4.0          6.0      -9.37352e-9
 5.41305e-8  6.0  -5.06122e-9  …  -7.9658e-9    6.0      -9.0834e-9 
 5.41305e-8  6.0  -5.06122e-9      8.0          6.0      -9.53876e-9
 5.41305e-8  6.0  -5.06122e-9     -8.84818e-9   6.0      -8.62868e-9
 5.41305e-8  6.0  -5.06122e-9      8.0          6.0      -9.49727e-9
 5.41305e-

 835   2.8265505354e-07  5.0018707223749104e+01
Computation time: 5.015755891799927

Synchronous solve output is:
objective_value(pb, y_sync) = 50.018707223749104


### Solving with Parallel Randomized Progressive Hedging

Randomized parallel and asynchronous solves leverage the distributed capacities of julia. In order to be used, workers need to be available. Local or remote workers can be added with [`addprocs`](https://docs.julialang.org/en/v1/stdlib/Distributed/#Distributed.addprocs).

`RPH` and `JuMP` packages need to be available for all workers, along with the scenario object and objective build function.

In [13]:
y_par = solve_randomized_par(pb, maxtime=5, printstep=50)
println("\nSynchronous solve output is:")
display(y_par)
@show objective_value(pb, y_par);

--------------------------------------------------------
--- Randomized Progressive Hedging - parallel
--------------------------------------------------------
Available workers: 3
   it   residual            objective                
Initialisation... done
   16   0.0000000000e+00    5.4372518198585304e+01    
   50   3.4300877513e-01    5.1970003026474288e+01 
  100   4.2721016964e-01    5.0653140575604660e+01 
  150   5.3532304692e-02    5.0206638445428624e+01 
  200   3.5282047697e-06    5.0060592275968204e+01 
  250   5.3143982090e-07    5.0019009798812604e+01 
  300   1.5790804599e-06    5.0000036187822168e+01 
  350   5.9718193080e-07    4.9999999118880304e+01 
  400   1.4137885860e-06    4.9999999325176795e+01 
  450   1.0098662720e-06    4.9999999148876071e+01 
  500   7.5652484843e-07    4.9999999543419754e+01 
  550   8.5799604077e-07    4.9999999418787112e+01 
  600   5.5863016540e-07    4.9999999399819167e+01 


16×15 Array{Float64,2}:
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7  …  -9.37352e-9   2.0   4.0       
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7      4.0          6.0  -9.77273e-9
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7     -9.06233e-9   6.0  -8.19998e-9
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7      8.0          6.0  -9.48558e-9
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7     -9.37352e-9   2.0   4.0       
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7  …   4.0          6.0  -9.37352e-9
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7      4.0          6.0  -9.37352e-9
 5.33861e-8  6.0  -1.40142e-9  1.96588e-7      8.0         10.0  -9.49882e-9
 5.33861e-8  6.0  -1.40142e-9  4.0            -9.37352e-9   2.0   4.0       
 5.33861e-8  6.0  -1.40142e-9  4.0             4.0          6.0  -9.37352e-9
 5.33861e-8  6.0  -1.40142e-9  4.0         …   4.41701e-7   6.0  -9.37244e-9
 5.33861e-8  6.0  -1.40142e-9  4.0             8.0          6.0  -9.49729e-9
 5.33861e-8  6.0  -1.40142e-9  4.0            -8.954

  616   5.5863016540e-07    4.9999999359246921e+01
Computation time: 5.016895055770874

Synchronous solve output is:
objective_value(pb, y_par) = 49.99999935924692


### Solving with Asynchronous Randomized Progressive Hedging

In [14]:
y_async = solve_randomized_async(pb, maxtime=5, printstep=100)
println("Asynchronous solve output is:")
display(y_async)
@show objective_value(pb, y_par)

--------------------------------------------------------
--- Randomized Progressive Hedging - asynchronous
--------------------------------------------------------
Available workers: 3
   it   residual            objective                 τ    delay
Initialisation... done
   16   0.0000000000e+00    5.4372518198585304e+01    4    0
  100   3.2729652479e-01    5.5198752823922618e+01    6    4
  200   2.3437571563e-01    5.2144478712234097e+01    6    2
  300   1.6498054308e-02    5.1208219585753866e+01    6    3
  400   9.9206220932e-04    5.1038148796322616e+01    9    3
  500   5.2957969811e-05    5.0892332932903614e+01    9    3
  600   1.4142074526e-05    5.0804872828472035e+01    9    3
  700   1.2189460535e-06    5.0622132637525837e+01    9    3
  800   5.9521018394e-02    5.0528162338799639e+01    9    7
  900   1.4288046844e-01    5.0391566180295413e+01    9    4
 1000   1.3984316112e-01    5.0225568887969189e+01    9    2
 1100   2.7548917887e-09    5.0125685317953426e+01    9 

16×15 Array{Float64,2}:
 5.20547e-8  6.0  -7.25972e-9  …  -9.77273e-9   2.0       4.0       
 5.20547e-8  6.0  -7.25972e-9      4.0          6.0      -9.77273e-9
 5.20547e-8  6.0  -7.25972e-9     -7.14317e-9   6.0      -9.19358e-9
 5.20547e-8  6.0  -7.25972e-9      8.0          6.0      -9.48291e-9
 5.20547e-8  6.0  -7.25972e-9     -9.37352e-9   2.0       4.0       
 5.20547e-8  6.0  -7.25972e-9  …   4.0          6.0      -9.37352e-9
 5.20547e-8  6.0  -7.25972e-9      3.30667      6.0      -9.3983e-9 
 5.20547e-8  6.0  -7.25972e-9      8.0          9.31692  -9.49974e-9
 5.20547e-8  6.0  -7.25972e-9     -9.37352e-9   2.0       4.0       
 5.20547e-8  6.0  -7.25972e-9      4.0          6.0      -9.37352e-9
 5.20547e-8  6.0  -7.25972e-9  …  -3.4347e-9    6.0      -9.31066e-9
 5.20547e-8  6.0  -7.25972e-9      8.0          6.0      -9.48191e-9
 5.20547e-8  6.0  -7.25972e-9     -8.55733e-9   6.0       1.42184e-8
 5.20547e-8  6.0  -7.25972e-9      8.0          6.0      -9.50297e-9
 5.20547e-

Asynchronous solve output is:
objective_value(pb, y_par) = 49.99999935924692


49.99999935924692